# Creating & Optimizing Equity Portfolios (without rebalancing)

## Getting started

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8")

In [ ]:
prices = pd.read_csv("stocks.csv", index_col = "Date", parse_dates = ["Date"])
prices

In [ ]:
returns = prices.pct_change().dropna()
returns

In [ ]:
returns.info()

In [ ]:
def ann_risk_return(returns_df): # assumes simple returns as input
    summary = pd.DataFrame(index = returns_df.columns)
    summary["ann. Risk"] = returns_df.std() * np.sqrt(252)
    log_returns = np.log(returns_df + 1)
    summary["CAGR"] = np.exp(log_returns.mean() * 252) - 1
    return summary

In [ ]:
summary = ann_risk_return(returns.iloc[1:])
summary

## Creating Random Portfolios (Part 1)

In [ ]:
returns

In [ ]:
noa = len(returns.columns) # number of assets
noa

In [ ]:
nop = 10000 # number of random portfolios
nop

In [ ]:
# 80,000 random floats between 0 and 1
np.random.seed(123)
matrix = np.random.random(noa * nop).reshape(nop, noa)

In [ ]:
matrix

In [ ]:
matrix.shape

In [ ]:
matrix.sum(axis = 1, keepdims= True)

In [ ]:
weights = matrix / matrix.sum(axis = 1, keepdims= True) # make portfolio weights summing up to 1
weights

In [ ]:
weights.sum(axis = 1, keepdims= True)

__one Portfolio__

In [ ]:
w1 = weights[0]
w1

__Weights are initial weights. No rebalancing thereafter.__

In [ ]:
returns.add(1).cumprod().mul(w1).sum(axis = 1) # weighted average investment multiple over time

In [ ]:
port1 = returns.add(1).cumprod().dot(w1) # alternatively: dot product
port1

In [ ]:
initial_w = returns.add(1).cumprod().mul(w1).iloc[0]
initial_w

In [ ]:
final_w = returns.add(1).cumprod().mul(w1).iloc[-1]
final_w / final_w.sum()

-> __no active rebalancing__! __Weights deviate__ from initial/target weights as prices move. <br>
-> weights of __outperforming__ (underperforming) Stocks __increase__ (decrease) over time. (__Momentum Trading Strategy__)


In [ ]:
ann_risk_return((port1 / port1.shift() - 1).to_frame())

## Creating Random Portfolios (Part 2)

__many Portfolios__

In [ ]:
weights

In [ ]:
ports = returns.add(1).cumprod().dot(weights.T)
ports # normalized prices for 10,000 portfolios

In [ ]:
port_ret = ports.pct_change().dropna()
port_ret # simple returns

In [ ]:
port_summary = ann_risk_return(port_ret)
port_summary

In [ ]:
summary

In [ ]:
plt.figure(figsize = (15, 9))
plt.scatter(port_summary.loc[:, "ann. Risk"], port_summary.loc[:, "CAGR"],s= 20, color = "red")
plt.scatter(summary.loc[:, "ann. Risk"], summary.loc[:, "CAGR"], s= 50, color = "black", marker = "D")
for i in summary.index:
    plt.annotate(i, xy=(summary.loc[i, "ann. Risk"]+0.01, summary.loc[i, "CAGR"]+0.01), size = 15)
plt.xlabel("ann. Risk (std)", fontsize = 15)
plt.ylabel("CAGR", fontsize = 15)
plt.title("Risk/Return", fontsize = 20)
plt.show()

## Performance Measurement: Risk-adjusted Return

__Risk-adjusted-Return (RaR): CAGR per unit of Risk. (similar to Sharpe Ratio)__

In [ ]:
summary["RaR"] = summary["CAGR"].div(summary["ann. Risk"])
summary

In [ ]:
port_summary["RaR"] = port_summary["CAGR"].div(port_summary["ann. Risk"])

In [ ]:
port_summary.sort_values("RaR")

In [ ]:
vmin = port_summary.RaR.min()
vmin

In [ ]:
vmax = port_summary.RaR.max()
vmax

In [ ]:
plt.figure(figsize = (15, 8))
plt.scatter(port_summary.loc[:, "ann. Risk"], port_summary.loc[:, "CAGR"], s= 20, 
            c = port_summary.loc[:, "RaR"], cmap = "coolwarm", vmin = vmin, vmax = vmax, alpha = 0.66)
plt.colorbar()
plt.scatter(summary.loc[:, "ann. Risk"], summary.loc[:, "CAGR"],s= 50, marker = "D", c = "black")
for i in summary.index:
    plt.annotate(i, xy=(summary.loc[i, "ann. Risk"]+0.01, summary.loc[i, "CAGR"]+0.01), size = 15)
plt.xlabel("ann. Risk (std)", fontsize = 15)
plt.ylabel("CAGR", fontsize = 15)
plt.title("Risk-adjusted Return", fontsize = 20)
plt.show()

## Portfolio Optimization

__Note: We are optimizing the past here (backward looking)!__

In [ ]:
import scipy.optimize as sco
pd.options.display.float_format = '{:.4f}'.format
np.set_printoptions(suppress = True)

In [ ]:
returns 

In [ ]:
#calculate portfolio CAGR (based on weights)
def port_ret(weights):
    simple_ret = returns.add(1).cumprod().dot(weights.T).pct_change()
    log_returns = np.log(simple_ret + 1)
    cagr = np.exp(log_returns.mean() * 252) - 1
    return cagr

In [ ]:
#calculate annualized portfolio volatility (based on weights)
def port_vol(weights):
    simple_ret = returns.add(1).cumprod().dot(weights.T).pct_change()
    return simple_ret.std() * np.sqrt(252)

In [ ]:
#define function to be minimized (sco only supports minimize, not maximize)
#-> maximize RaR == minimize RaR * (-1)
def min_func_RaR(weights): 
     return -(port_ret(weights)) / port_vol(weights) #Risk-adjusted Return * (-1)

In [ ]:
#number of assets
noa = len(returns.columns)
noa

In [ ]:
#equal weights (starting point of optimization)
eweigths = np.full(noa, 1/noa)
eweigths

In [ ]:
#constraint: weights must sum up to 1 -> sum of weights - 1 = 0
cons = ({"type": "eq", "fun": lambda x: np.sum(x) - 1})

In [ ]:
#bounds: all weights shall be between 0 and 1 -> can be changed
bnds =  tuple((0, 1) for x in range(noa))
bnds

In [ ]:
#run optimization based on function to be minimized, starting with equal weights and based on respective bounds and constraints
opts = sco.minimize(min_func_RaR, eweigths, method = "SLSQP", bounds = bnds, constraints= cons)

In [ ]:
#output of optimization
opts

In [ ]:
#getting the optimal weights
optimal_weights = opts["x"]
optimal_weights

In [ ]:
pd.Series(data = optimal_weights, index = returns.columns).sort_values(ascending = False).head(20)

-> __Optimization (without bounds) does not necessarily lead to practical/factual diversification.__

In [ ]:
#cagr of the optimal portfolio
cagr_opt = port_ret(optimal_weights)
cagr_opt

In [ ]:
#volatility of the optimal portfolio
vol_opt = port_vol(optimal_weights)
vol_opt

In [ ]:
#RaR of the optimal portfolio
RaR_opt = -min_func_RaR(optimal_weights)
RaR_opt

In [ ]:
plt.figure(figsize = (15, 8))
plt.scatter(port_summary.loc[:, "ann. Risk"], port_summary.loc[:, "CAGR"], s= 20, 
            c = port_summary.loc[:, "RaR"], cmap = "coolwarm", vmin = vmin, vmax = vmax, alpha = 0.66)
plt.colorbar()
plt.scatter(summary.loc[:, "ann. Risk"], summary.loc[:, "CAGR"],s= 50, marker = "D", c = "black")
for i in summary.index:
    plt.annotate(i, xy=(summary.loc[i, "ann. Risk"]+0.01, summary.loc[i, "CAGR"]+0.01), size = 15)
plt.scatter(x = vol_opt, y = cagr_opt, s = 100, marker = "X", c = "purple") # best Portfolio
plt.xlabel("ann. Risk (std)", fontsize = 15)
plt.ylabel("CAGR", fontsize = 15)
plt.title("The optimal Portfolio", fontsize = 20)
plt.show()

__Reminder: We are optimizing the past here (backward looking).__ <br>
-> Very unlikely we had selected this optimal portfolio back in 2017 (__look ahead bias__)! <br>
-> Very unlikely this will be the optimal portfolio in the future (__past performance is not a good indicator for future performance__)!

## Minimum Variance Portfolio

(use code above with following replacements:)

In [ ]:
# Minimum Variance Portfolio
opts = sco.minimize(port_vol, eweigths, method = "SLSQP", bounds = bnds, constraints= cons)

## Maximum Return Portfolio

(use code above with following replacements:)

In [ ]:
#calculate portfolio CAGR (based on weights)
def port_ret(weights):
    simple_ret = returns.add(1).cumprod().dot(weights.T).pct_change()
    log_returns = np.log(simple_ret + 1)
    cagr = np.exp(log_returns.mean() * 252) - 1
    return cagr * (-1)

In [ ]:
# Maximum Return Portfolio
opts = sco.minimize(port_ret, eweigths, method = "SLSQP", bounds = bnds, constraints= cons)

In [ ]:
#cagr of the optimal portfolio
cagr_opt = -port_ret(optimal_weights)
cagr_opt

## The Efficient Frontier

__Idea: Find for each return level the portfolio with the lowest volatility.__

In [ ]:
returns

In [ ]:
#calculate portfolio CAGR (based on weights)
def port_ret(weights):
    simple_ret = returns.add(1).cumprod().dot(weights.T).pct_change()
    log_returns = np.log(simple_ret + 1)
    cagr = np.exp(log_returns.mean() * 252) - 1
    return cagr

In [ ]:
#calculate annualized portfolio volatility (based on weights)
def port_vol(weights):
    simple_ret = returns.add(1).cumprod().dot(weights.T).pct_change()
    return simple_ret.std() * np.sqrt(252)

In [ ]:
noa = len(returns.columns)
noa

In [ ]:
#equal weights (starting point of optimization)
eweigths = np.full(noa, 1/noa)
eweigths

In [ ]:
summary

In [ ]:
tcagrs = np.linspace(summary.CAGR.min(), summary.CAGR.max(), 100)
tcagrs # evenly spaced target returns between min and max

In [ ]:
# portfolio return == tcagr
cons = ({"type": "eq", "fun": lambda x: port_ret(x) - tcagr},
       {"type": "eq", "fun": lambda x: np.sum(x) - 1})

In [ ]:
bnds =  tuple((0, 1) for x in range(noa))
bnds

In [ ]:
vols = []

In [ ]:
#for each target return, find the portfolio with the lowest volatility
for tcagr in tcagrs:
    res = sco.minimize(port_vol, eweigths, method = "SLSQP", bounds = bnds, constraints = cons)
    vols.append(res["fun"])
vols = np.array(vols)

In [ ]:
vols

In [ ]:
plt.figure(figsize = (15, 8))
plt.scatter(port_summary.loc[:, "ann. Risk"], port_summary.loc[:, "CAGR"], s= 20, 
            c = port_summary.loc[:, "RaR"], cmap = "coolwarm", vmin = vmin, vmax = vmax, alpha = 0.66)
plt.colorbar()
plt.scatter(summary.loc[:, "ann. Risk"], summary.loc[:, "CAGR"],s= 50, marker = "D", c = "black")
for i in summary.index:
    plt.annotate(i, xy=(summary.loc[i, "ann. Risk"]+0.01, summary.loc[i, "CAGR"]+0.01), size = 15)
plt.plot(vols, tcagrs) # efficient frontier
plt.xlabel("ann. Risk (std)", fontsize = 15)
plt.ylabel("CAGR", fontsize = 15)
plt.title("The Efficient Frontier", fontsize = 20)
plt.show()